# MEMBRANE PROTEIN SIMULATION: GLIC
Setting up membrane protein simulations is trickier than preparing a soluble protein. However the basic needs are the same. Meaning that you need coordinate files (protein and lipid) and the corresponding topologies. Forcefield files and the topology of lipids is provided within the tuturial files you will download below. The tutorial is a bit long. But I wanted to explain the steps to a person who is not familiar with the process at all. 

Before we start lets create a project directory named GLIC_pH70. The protein we are going to use, as the name of the folder states, is called GLIC. GLIC is a pH-gated cationic channel, homologous to cys-loop receptors. pH-gated here means the activation is done by increasing the pH from 7.0 to 4.0. There for we named the folder pH70. After you have gone through this tutorial and enjoyed it, you can try changing the protonation states of some residues and setup another simulation as a practice. Back to business...

Now within GLIC_pH70 create the following folders: *prep, run-files, em, equi, prod*. *prep* folder is our working directory to setup the simulation box. *run-files* is the directy we will place the final files. In theory this is just a duplication of the files within *prep* but things gets messy there. So it is a good practice to have a clean copy of the basic files in a seperate folder. *em, equi,* and *prod* folders will contain the minimisation, equilibration and the production runs. 

#### PREREQUISITES
Even if setting up simulations is not your main job, you clearly felt the need to setup a simulation or two that you have ended up going through this page. I strongly recommend getting familiar with the basic terminal commands. If you are an experimentalit, it might be a scary transition. But the tasks are much easier to complete!!! Trust me I have done that transition. There is nothing to fear. Just don't use the remove/move commands with a *. You are more than welcomed to contact me for help.

Also I assume you know the meaning of a simulation. If not, I do not recemmonned starting with membrane proteins. Setting up a membrane protein simulation right away could be frusturating due to extra steps required to prepare. [Justin Lemkul's tutorials](http://www.mdtutorials.com/) are a perfect way to start.

Now the real requirements: You should have VMD and GROMACS installed. If you are using terminal (bash) set the variables below and source the gromacs if it is not in your path:

In [ ]:
#This is a python cell, the rest of the cells are bash. Do not run this cell.
#If you are on terminal use the commented out cells below for setting the environment.
#export VMD=/path/to/your/vmd_executable
#source /path/to/your/gmx/bin/GMXRC 

In [1]:
%%bash
#If you are running this as a jupyter notebook run the command below to set the aliases.
#You must also uncomment the lines on top of each cell
#echo 'export VMD="/path/to/your/vmd_executable"' >> sources.sh
#echo 'source /path/to/your/gmx/bin/GMXRC' >> sources.sh
echo "export VMD=/Applications/VMD1.9.2.app/Contents/vmd/vmd_MACOSXX86" >> sources.sh
echo "source /usr/local/gromacs/bin/GMXRC" >> sources.sh

## PREPARE THE COORDINATES AND THE TOPOLOGY 

Go a head and [download the tutorial files](https://github.com/ozyo/stb_proteinart/blob/master/tutorial/GLIC/GLIC_Tutorial_gmx2016.tar.gz) and extract. Within the extracted folder you will find two forcefield folders. It's better if you don't make modifications to the main gromacs forcefield folder in the beginning. So keep these folders here for now, even tough one of those folder is a duplication of what is already installed. Move them over to the prep folder. Both of them are needed! One is the original amber99sb-ILDN folder. The other one is extended with the lipid parameters.

I have provided two coordinate files ending with .pdb. 4NPQ_BA1.pdb is the first biological assembly of the GLIC 4NPQ structure. Crystal structures often contain other molecules than protein or missing residues/atoms. I have cleaned the crystal structure from extra molecules and other biological assemblies. Then used MODELLER to fix the missing backbone&sidechain atoms. There are other ways to fix the structure. But these will not be covered in this tutorial. The second coordinate file contains Berger POPC lipids. It is a bilayer with 332 lipids. This lipid patch is just enough size to accomodate GLIC. Copy both files to prep folder. 

You will also find a .tcl script and gmembed.mdp, gmembed.dat files. Copy these to the prep folder too. I will explain their job below. There are some limitations when we use bash magic cells. You can run the commands in a terminal or close the notebook, move it to prep folder and open it again. This way you should be able to run the cells directly.

Now the real fun begins; load these two structures to your favorite visualization software. You see that these two structures are not aligned. In order to place the protein into the membrane first we need to align them. Our membrane patch expands on the xy plane. That means we need to align the transmembrane domain to its 3rd principal axis, z. 

If you are adventurous you can try to do this manually :-) If you want to a short cut here are some awesome tools from gromacs and a beatiful VMD script.
1. First we align the protein on its principal axes and center it at 0 0 0
2. Next we rotate it by 90&deg; around the y axis. These steps aligns the protein channel with the z axis. It also enables us to place the transmembrane domain to the bilayer correctly.
3. As the last step we move/translate the protein into the bilayer. The .tcl script moves the protein by aligning the the center of the trasmembrane domain to the center of the bilayer.

In [2]:
%%bash
source sources.sh

gmx editconf -f 4NPQ_BA1.pdb -o GLIC_princ.pdb -center 0 0 0 -princ <<EOF
1
EOF

gmx editconf -f GLIC_princ.pdb -o GLIC_princ_rotate.pdb -rotate 0 90 0

$VMD -dispdev text -e protein_center_to_lipids.tcl -args GLIC_princ_rotate.pdb popc_332.pdb GLIC_princ_rotate_translate

Read 12630 atoms
No velocities found
    system size : 11.743  8.491 11.801 (nm)
    center      : -5.427 -1.985 -0.176 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

Selected 1: 'Protein'
new system size : 11.374  8.379  8.278
    shift       :  5.426  1.985  0.173 (nm)
new center      : -0.000  0.000 -0.000 (nm)
new box vectors :  0.000  0.000  0.000 (nm)
new box angles  :   0.00   0.00   0.00 (degrees)
new box volume  :   0.00               (nm^3)
Read 12630 atoms
No velocities found
    system size : 11.375  8.379  8.278 (nm)
    center      : -0.000  0.000 -0.000 (nm)
    box vectors :  0.000  0.000 

          :-) GROMACS - gmx editconf, 2016.2-dev-20161028-2ed67b7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra    Gerrit Groenhof  
 Christoph Junghans   Anca Hamuraru    Vincent Hindriksen Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff    Erik Marklund   
   Teemu Murtola       Szilard Pall       Sander Pronk      Roland Schulz   
  Alexey Shvetsov     Michael Shirts     Alfons Sijbers     Peter Tieleman  
  Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala University, Stockholm

If you load the output generated with above commands to a visualisation software, you will be able to understand better what each steps does. 

Now we have our protein placed at the same plane as our bilayer. However they are still two seperate files. Before we concatanate them, we need to process the protein file further and obtain a topology file for it. For lipids we already have the topology files within the forcefield directory. *pdb2gmx* is the tool in GMX that creates the protein topologies for us. Since we want to run a GLIC simulation at pH 7.0, no need to assign different protonation states (protonation states of histidines are tricky but we will go with default assignments for now). 
pdb2gmx will ask for (1) a forcefield choice and (2) a water model. When prompted choose 1 (corresponds to amber99s-ildn_berger forcefield) and 1 (TIP3P water model).

In [3]:
%%bash
source sources.sh

gmx pdb2gmx -f GLIC_princ_rotate_translate.pdb -o GLIC_pH70.pdb -ignh -vsite hydrogens <<EOF
1
1
EOF


Select the Force Field:
From current directory:
 1: AMBER99SB-ILDN force field, extended with Berger united-atom lipids
 2: AMBER99SB-ILDN protein, nucleic AMBER94 (Lindorff-Larsen et al., Proteins 78, 1950-58, 2010)
From '/usr/local/gromacs/share/gromacs/top':
 3: AMBER03 protein, nucleic AMBER94 (Duan et al., J. Comp. Chem. 24, 1999-2012, 2003)
 4: AMBER94 force field (Cornell et al., JACS 117, 5179-5197, 1995)
 5: AMBER96 protein, nucleic AMBER94 (Kollman et al., Acc. Chem. Res. 29, 461-469, 1996)
 6: AMBER99 protein, nucleic AMBER94 (Wang et al., J. Comp. Chem. 21, 1049-1074, 2000)
 7: AMBER99SB protein, nucleic AMBER94 (Hornak et al., Proteins 65, 712-725, 2006)
 8: AMBER99SB-ILDN protein, nucleic AMBER94 (Lindorff-Larsen et al., Proteins 78, 1950-58, 2010)
 9: AMBERGS force field (Garcia & Sanbonmatsu, PNAS 99, 2782-2787, 2002)
10: CHARMM27 all-atom force field (CHARM22 plus CMAP for proteins)
11: CHARMM36 all-atom force field (March 2017)
12: GROMOS96 43a1 force field
13: GROMO

           :-) GROMACS - gmx pdb2gmx, 2016.2-dev-20161028-2ed67b7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra    Gerrit Groenhof  
 Christoph Junghans   Anca Hamuraru    Vincent Hindriksen Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff    Erik Marklund   
   Teemu Murtola       Szilard Pall       Sander Pronk      Roland Schulz   
  Alexey Shvetsov     Michael Shirts     Alfons Sijbers     Peter Tieleman  
  Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala University, Stockholm

Note that in the above command we have ignored the hydrogens within our input structure (ignh) and added new hydrogens along with virtual sites. Virtual sites are required to speed up the calculations. You can read more about it on the gromacs manual. But in simple words, it creates a ghost particle which represents the hydrogens together with heavy atoms. By doing so, we don't have to calculate the fast vibrations of hydrogens in small intervals (timestep). 

So now we have a protein structure that can be combined with the lipid file since it has the correct terminal ends

In [4]:
%%bash

head -4 popc_332.pdb > GLIC_pH70_popc.pdb
grep 'ATOM' GLIC_pH70.pdb >> GLIC_pH70_popc.pdb
grep 'POPC' popc_332.pdb >> GLIC_pH70_popc.pdb
echo 'END' >> GLIC_pH70_popc.pdb

Next we create a box and solvate it. The bilayer file I have provided already contains the box dimensions required. So all we need to do is to center the system within that box and add waters. Adding waters to bilayer systems are a bit tricky, especially with channels. You want water within the channel, but you not within the bilayer. Since there is space within the bilayer, solvation tools fills it with waters. But luckily there are several work arounds to the issue. I will use the method where we increase the vdW radius of the POPC carbons.

In [5]:
%%bash
source sources.sh

gmx editconf -f GLIC_pH70_popc.pdb -o GLIC_pH70_popc_center.pdb -c -bt cubic
cp /usr/local/gromacs/share/gromacs/top/vdwradii.dat b_vdwradii.dat
sed '/; Water charge sites/ a\
POP  C     0.45 \
' b_vdwradii.dat > vdwradii.dat
gmx solvate -cp GLIC_pH70_popc_center.pdb -cs -o GLIC_pH70_popc_center_sol.pdb
rm vdwradii.dat

Read 45034 atoms
Volume: 1758.73 nm^3, corresponds to roughly 791400 electrons
No velocities found
    system size : 12.933 12.566 11.659 (nm)
    diameter    : 16.346               (nm)
    center      :  5.165  5.272  6.341 (nm)
    box vectors : 10.484 10.484 16.000 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :1758.73               (nm^3)
    shift       :  0.077 -0.029  1.659 (nm)
new center      :  5.242  5.242  8.000 (nm)

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volume

          :-) GROMACS - gmx editconf, 2016.2-dev-20161028-2ed67b7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra    Gerrit Groenhof  
 Christoph Junghans   Anca Hamuraru    Vincent Hindriksen Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff    Erik Marklund   
   Teemu Murtola       Szilard Pall       Sander Pronk      Roland Schulz   
  Alexey Shvetsov     Michael Shirts     Alfons Sijbers     Peter Tieleman  
  Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala University, Stockholm

Now take a look at the end file GLIC_pH70_popc_center_sol.pdb and make sure that the bilayer is not invaded by a ton of water molecules. Altough above method worked for me everytime, it is better to make sure then cry over a setup not working. You can play around with the radius *0.45* to get better results.

At this point we still have overlapping lipids with the protein. So if you choose to look at it with cartoon representation you will not see parts of the protein. Do not be alarmed. The protein is there! Change the representation to lines/licorice or speheres then you will see it.

If everything looks ok proceed to run g_membed to remove overlapping molecules. Since g_membed uses mdrun, we need to update the topology file to include lipids and water. After g_membed run we add more waters to the system so that the pore area previously occupied by lipids also gets hydrated. Running g_membed will take a while, don't worry if it doesn't finish right away. You can keep track of the process by going into the g_membed folder, and checking the log files.

In [6]:
%%bash
source sources.sh

SOL_NR=`grep "OW" GLIC_pH70_popc_center_sol.pdb | wc | awk '{printf $1}'`
LIPID_NR=`grep "POPC" GLIC_pH70_popc_center_sol.pdb | wc | awk '{printf $1/52}'`

grep -v "Protein_chain_[B-E]" topol.top > GLIC_pH70_popc_sol.top
sed -i '' '/topol_Protein_chain_A.itp"/ a\
#include "amber99sb-ildn-berger.ff/popc.itp" \
' GLIC_pH70_popc_sol.top
sed -i '' 's/Protein_chain_A     1/Protein_chain_A     5/g' GLIC_pH70_popc_sol.top
echo 'POPC            '$LIPID_NR >> GLIC_pH70_popc_sol.top
echo 'SOL             '$SOL_NR >> GLIC_pH70_popc_sol.top
sed -i '' 's/amber99sb-ildn.ff/amber99sb-ildn-berger.ff/g' GLIC_pH70_popc_sol.top

mkdir -p g_membed
cp gmembed.mdp gmembed.dat g_membed/
cd g_membed
rm *.tpr *.pdb
touch box.mdp 
gmx grompp -f box.mdp -c ../GLIC_pH70_popc_center_sol.pdb -p ../GLIC_pH70_popc_sol.top -o b4membed.tpr
gmx trjconv -f ../GLIC_pH70_popc_center_sol.pdb -o b4membed.pdb -s b4membed -ur rect -pbc mol<<EOF
0
EOF

gmx make_ndx -f ../GLIC_pH70_popc_center_sol.pdb -o index_before_gmembed.ndx<<EOF
name 13 POPC
q
EOF

gmx grompp -f gmembed.mdp -c b4membed.pdb -o gmembed.tpr -maxwarn 1 -n index_before_gmembed -p ../GLIC_pH70_popc_sol.top -v

gmx mdrun -nt 1 -s gmembed.tpr -membed gmembed.dat -c GLIC_pH70_popc_further_solvate.pdb -mn index_before_gmembed -v <<EOF
1
13
EOF

cp /usr/local/gromacs/share/gromacs/top/vdwradii.dat b_vdwradii.dat
sed '/; Water charge sites/ a\
POP  C     0.45 \
' b_vdwradii.dat > vdwradii.dat

gmx solvate -cp GLIC_pH70_popc_further_solvate.pdb -cs -o GLIC_pH70_POPC_SOL.pdb
rm vdwradii.dat

Analysing residue names:
There are:  1555    Protein residues
There are:   332      Other residues
There are: 34403      Water residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 11 Mb of data
Select group for output
Selected 0: 'System'
Going to read 0 old index file(s)
Analysing residue names:
There are:  1555    Protein residues
There are:   332      Other residues
There are: 34403      Water residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 148243 atoms
  1 Protein             : 27770 atoms
  2 Protein-H           : 15030 atoms
  3 C-alpha             :  1555 atoms
  4 Backbone            :  4665 atoms
  5 MainChain           :  6225 atoms
  6 MainChain+Cb        :  7750 atoms
  7 MainChain+H         :  7700 atoms
  8 SideChain           : 20070 atoms
  9 SideChain-H         :  8805 atoms
 10 

           :-) GROMACS - gmx grompp, 2016.2-dev-20161028-2ed67b7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra    Gerrit Groenhof  
 Christoph Junghans   Anca Hamuraru    Vincent Hindriksen Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff    Erik Marklund   
   Teemu Murtola       Szilard Pall       Sander Pronk      Roland Schulz   
  Alexey Shvetsov     Michael Shirts     Alfons Sijbers     Peter Tieleman  
  Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala University, Stockholm 

Next step is to update the topology with the new number of lipids and water molecules. After that we will place ions to our system using ionic strength of 0.1M . When adding ions one can use the box volume, however membrane proteins such as GLIC contain a huge extracellular domain that cuts your solvent area by half, at least. So instead I am using molality to determine the number of ions. We also need to know the charge of the protein so that we can neutralize it. It is important that the system has a neutral charge beucase we are using an alogrithm that makes this assumption when calculation the electrostatic forces.

You would see the charge of the protein when you run pdb2gmx. But if you haven't been paying attention to the terminal output at that step you have no idea, even if you were looking it can be overwhelming. When setting up a new system it is a good practice to examine the printed info from pdb2gmx, not just for system charge but for disulfide bridge formations, protonation states on histidines, number of chains/residues, etc. But don't worry we won't go back. There is a neat way of finding the total system charge with the commands below, you can use them for automating the setup later on.

In [7]:
%%bash
source sources.sh

#Variables between bash magic cells are not transferable. These are not necessary when running from terminal.
SOL_NR=`grep "OW" GLIC_pH70_popc_center_sol.pdb | wc | awk '{printf $1}'`
LIPID_NR=`grep "POPC" GLIC_pH70_popc_center_sol.pdb | wc | awk '{printf $1/52}'`

#This is only needed since each bash cell starts from where you are running the notebook. 
#If you are running directly from terminal you should be there already.
cd g_membed

SOL_NR2=`grep "OW" GLIC_pH70_POPC_SOL.pdb | wc | awk '{printf $1}'`
LIPID_NR2=`grep "POPC" GLIC_pH70_POPC_SOL.pdb | wc | awk '{printf $1/52}'`

cp ../GLIC_pH70_popc_sol.top GLIC_pH70_POPC_SOL.top
sed -i '' "s/$SOL_NR/$SOL_NR2/g" GLIC_pH70_POPC_SOL.top
sed -i '' "s/$LIPID_NR/$LIPID_NR2/g" GLIC_pH70_POPC_SOL.top

SYS_CHARGE=`grep qtot ../topol_Protein_chain_A.itp | tail -1 | awk '{print $11 * 5}'`

echo $SYS_CHARGE
ION_NUMBER=`echo "scale=9; (0.1 / 55.5)*$SOL_NR2" | bc `
ION_NUMBER=${ION_NUMBER%%.*}

if [ $SYS_CHARGE -gt 0 ]; then
        let NEG_ION=$ION_NUMBER+$SYS_CHARGE
        let POS_ION=$ION_NUMBER
        echo "CL- :"$NEG_ION" Na+ :"$POS_ION
elif [ $SYS_CHARGE -lt 0 ]; then
        let NEG_ION=$ION_NUMBER
        let POS_ION=$ION_NUMBER-$SYS_CHARGE
        echo "CL- :"$NEG_ION" Na+ :"$POS_ION
else
        let NEG_ION=$ION_NUMBER
        let POS_ION=$ION_NUMBER
        echo "CL- :"$NEG_ION" Na+ :"$POS_ION
fi

cd ../
cp g_membed/GLIC_pH70_POPC_SOL.top GLIC_pH70_POPC_SOL_ION.top
touch ion.mdp
gmx grompp -f ion.mdp -c g_membed/GLIC_pH70_POPC_SOL.pdb -p GLIC_pH70_POPC_SOL_ION.top -o 4ion.tpr
gmx genion -s 4ion.tpr -nn $NEG_ION -np $POS_ION -p GLIC_pH70_POPC_SOL_ION.top -o GLIC_pH70_POPC_SOL_ION.pdb<<EOF
15
EOF


-30
CL- :69 Na+ :99
Analysing residue names:
There are:  1555    Protein residues
There are:   306      Other residues
There are: 38555      Water residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 12 Mb of data
Will try to add 99 NA ions and 69 CL ions.
Select a continuous group of solvent molecules
Selected 15: 'SOL'

Processing topology
Replacing 168 solute molecules in topology file (GLIC_pH70_POPC_SOL_ION.top)  by 99 NA and 69 CL ions.


           :-) GROMACS - gmx grompp, 2016.2-dev-20161028-2ed67b7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra    Gerrit Groenhof  
 Christoph Junghans   Anca Hamuraru    Vincent Hindriksen Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff    Erik Marklund   
   Teemu Murtola       Szilard Pall       Sander Pronk      Roland Schulz   
  Alexey Shvetsov     Michael Shirts     Alfons Sijbers     Peter Tieleman  
  Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala University, Stockholm 

Now we have a coordinate system and a topology file ready. And we are almost done with our preperations. Membrane protein systems might require a bit longer equilibration as we have a heterogeneous system. To prevent protein changing its conformation abruptly during the equilibration, we will place position restraints on all the atoms except the hydrogens. These restraints will then be gradually reduced to only backbone and the Calpha atoms of the proteins. For GMX to apply these restraints we have to create those files. Run the commands below to do just that. 

In [8]:
%%bash
source sources.sh

echo '; Include Position restraint file
#ifdef POSRES_HEAVY
#include "posre_Protein_chain_A.itp"
#endif
                                                                                                                                                                                  
; Include Position restraint file
#ifdef POSRES_BACKBONE
#include "posre_Protein_chain_A_backbone.itp"
#endif
                                                                                                                                                                                  
; Include Position restraint file                                                                                                                                                 
#ifdef POSRES_CA
#include "posre_Protein_chain_A_ca.itp"
#endif' >> topol_Protein_chain_A.itp

gmx make_ndx -f GLIC_pH70_POPC_SOL_ION.pdb -o posre.ndx<<EOF
keep 4
0 & chain A 
0 & chain A & a CA
q
EOF

gmx genrestr -f GLIC_pH70_POPC_SOL_ION.pdb -n posre.ndx -o posre_Protein_chain_A_backbone<<EOF
1
EOF

gmx genrestr -f GLIC_pH70_POPC_SOL_ION.pdb -n posre.ndx -o posre_Protein_chain_A_ca<<EOF
2
EOF

cp *chain_A*itp ../run-files/
cp GLIC_pH70_POPC_SOL_ION.??? ../run-files/
cp -r amber99sb*.ff ../run-files/

Going to read 0 old index file(s)
Analysing residue names:
There are:  1555    Protein residues
There are:   306      Other residues
There are: 38387      Water residues
There are:   168        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 159011 atoms
  1 Protein             : 27770 atoms
  2 Protein-H           : 15030 atoms
  3 C-alpha             :  1555 atoms
  4 Backbone            :  4665 atoms
  5 MainChain           :  6225 atoms
  6 MainChain+Cb        :  7750 atoms
  7 MainChain+H         :  7700 atoms
  8 SideChain           : 20070 atoms
  9 SideChain-H         :  8805 atoms
 10 Prot-Masses         : 25370 atoms
 11 non-Protein         : 131241 atoms
 12 Other               : 15912 atoms
 13 POPC                : 15912 atoms
 14 NA                  :    99 atoms
 15 CL                 

          :-) GROMACS - gmx make_ndx, 2016.2-dev-20161028-2ed67b7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra    Gerrit Groenhof  
 Christoph Junghans   Anca Hamuraru    Vincent Hindriksen Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff    Erik Marklund   
   Teemu Murtola       Szilard Pall       Sander Pronk      Roland Schulz   
  Alexey Shvetsov     Michael Shirts     Alfons Sijbers     Peter Tieleman  
  Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala University, Stockholm

And we are done!